In [1]:
!pip3 install earthengine-api 


In [1]:
import ee # Google Earth Engine
import datetime

import glob

import time #sleep before screenshot

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
import folium
from folium import plugins
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
gdrive_project_pathname = '/My Drive/w210'  #@param {type: "string"}

In [3]:
import os.path
from google.colab import drive

# Mount the current user's Google Drive
GOOGLE_DRIVE_MOUNT_POINT = "/content/drive"
print("Mounting Google Drive beneath %s" % GOOGLE_DRIVE_MOUNT_POINT)
drive.mount(GOOGLE_DRIVE_MOUNT_POINT, force_remount = True)

# Build the pathname to the project's folder residing beneath the current user's Google Drive
if not gdrive_project_pathname.startswith("/"):
  gdrive_project_pathname = "/" + gdrive_project_pathname
abs_project_pathname = GOOGLE_DRIVE_MOUNT_POINT + gdrive_project_pathname
print("Project folder: %s" % gdrive_project_pathname)
print("Project folder for test_graphs: %s" % abs_project_pathname + '/test_graphs/')
files = glob.glob(abs_project_pathname + '/test_graphs' + '/*')
for f in files:
    print(f)

Mounting Google Drive beneath /content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Project folder: /My Drive/w210
Project folder for test_graphs: /content/drive/My Drive/w210/test_graphs/
/content/drive/My Drive/w210/test_graphs/LA Traffic.ipynb
/content/drive/My Drive/w210/test_graphs/test_graph.html
/content/drive/My Drive/w210/test_graphs/heatmap.html
/content/drive/My Drive/w210/test_graphs/Week10_presentation_prediction.html
/content/drive/My Drive/w210/test_graphs/GEE_SatelliteImagery_graphs

Initialize Google Earth Engine (GEE)

In [4]:
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

EEException: ignored

Read the data

In [13]:
!ls -l

total 8
drwx------ 4 root root 4096 Jul  5 09:45 drive
drwxr-xr-x 1 root root 4096 Jun 26 16:26 sample_data


In [5]:
### Read the data 
df=pd.read_csv(abs_project_pathname + "/Angela_files/Week10_presentation_prediction.csv")
df.head(25)

,Unnamed: 0,lat,lon,Class
0,4,24.300192,73.371114,Rainfed
1,5,24.300192,73.371383,Rainfed
2,6,24.300192,73.371653,Rainfed
3,7,24.300192,73.371922,Rainfed
4,8,24.300192,73.372192,Rainfed
5,9,24.300192,73.372461,Rainfed
6,13,24.300192,73.373539,Rainfed
7,14,24.300192,73.373809,Rainfed
8,15,24.300192,73.374078,Rainfed
9,16,24.300192,73.374347,Rainfed


Define the function to plot satellite images or pull satellite image data

In [6]:
# df['color']=df.apply(lambda row:"Orange" if row['GFSAD30']==1 else
#                                   "green" if row['GFSAD30']==2 else
#                                    "grey", axis=1)

df['color']=df.apply(lambda row:"Orange" if row['Class']=='Irrigated' else
                                  "green" if row['Class']=='Rainfed' else
                                  "black" if row['Class']=='Non-cropland' else
                                   "grey", axis=1)

In [12]:
df

,Unnamed: 0,lat,lon,Class,color
0,4,24.300192,73.371114,Rainfed,green
1,5,24.300192,73.371383,Rainfed,green
2,6,24.300192,73.371653,Rainfed,green
3,7,24.300192,73.371922,Rainfed,green
4,8,24.300192,73.372192,Rainfed,green
...,...,...,...,...,...
137636,137636,24.399905,73.468671,Non-cropland,black
137637,137637,24.399905,73.468940,Non-cropland,black
137638,137638,24.399905,73.469210,Non-cropland,black
137639,137639,24.399905,73.469479,Non-cropland,black


Create a map with latitude, longitude coordinates

In [9]:
m=folium.Map([24.35,73.42],zoom_start=12)
#location=location[0:2000]
for lat,lon,color in zip(df['lat'],df['lon'],df['color']):
     folium.Circle([lat, lon], radius = 10,
                            color='b',
                            fill=True,
                            fill_opacity=0.7,
                            fill_color=color,
                           ).add_to(m)
m.save(abs_project_pathname + '/test_graphs/Week10_presentation_prediction.html')

In [32]:
df_data = df[['lat', 'lon']].values

# plot heatmap
m.add_child(plugins.HeatMap(df_data, radius=10))
m.save(abs_project_pathname + '/test_graphs/heatmap.html')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  after removing the cwd from sys.path.


In [35]:
m=folium.Map([24.35,73.42],zoom_start=11)
for lat, lon, GFSAD30 in zip(df['lat'], df['lon'], df['GFSAD30']):
    folium.CircleMarker(
        [lat, lon],
        radius=.15*GFSAD30,
        color='b',
        threshold_scale=[0,1,2,3],
        fill_color=color,
        fill=True,
        fill_opacity=0.7
        ).add_to(m)
m